# 1. Numpy를 사용하여 RNN 동작 구현해보기

In [1]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

In [2]:
print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
Wx = np.random.random((hidden_size, input_size))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

- 임베딩 벡터의 차원이 256이고 문장의 길이가 10으로 고정되었고, 은닉상태의 차원이 128이다. 이때 RNN의 wh,wx,b 의 크기는 얼마인가?

- wx = 256 * 128
- wh = 128 * 128
- b = 128 

In [ ]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [ ]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0)
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.84715023 0.85185152 0.94789783 0.76488895 0.958207   0.84938779
  0.5088422  0.96349715]
 [0.99997134 0.99856298 0.99967925 0.99987787 0.99989994 0.99933066
  0.99965825 0.99999421]
 [0.99999082 0.99967378 0.99994559 0.99998852 0.99999002 0.99977909
  0.99995582 0.99999976]
 [0.99999391 0.99920695 0.99995371 0.99998315 0.99999162 0.99976157
  0.99995984 0.99999974]
 [0.99997461 0.99811734 0.99967025 0.99996771 0.99994798 0.99886396
  0.99990766 0.99999886]
 [0.9999905  0.99925815 0.99991277 0.9999808  0.99998462 0.99968082
  0.99994618 0.99999957]
 [0.99997944 0.99908483 0.99981222 0.99996777 0.99996263 0.99923153
  0.99990023 0.99999884]
 [0.99998835 0.99914391 0.99988498 0.99998704 0.99998384 0.999637
  0.99995734 0.99999973]
 [0.99999606 0.99967952 0.9999537  0.99998393 0.99999126 0.99991227
  0.9999582  0.99999967]
 [0.9999939  0.99964879 0.99994264 0.99999217 0.99999195 0.99987604
  0.99997125 0.99999986]]


PyTorch의 nn.GRU와 nn.RNN은 출력 측면에서 shape가 항상 동일합니다. 둘 다 아래와 같은 출력값을 반환합니다.

- output: 모든 time step에서의 hidden state 값들이 포함되어 있으며, shape는 (batch_size, seq_length, num_directions * hidden_size)입니다. 여기서 num_directions는 양방향 RNN 또는 GRU의 경우 2이고, 단방향 RNN 또는 GRU의 경우 1입니다.  

- hidden: 마지막 time step에서의 hidden state 값들이 포함되어 있으며, shape는 (num_layers * num_directions, batch_size, hidden_size)입니다.
nn.RNN과 nn.GRU 모두 동일한 출력 shape를 가지지만, 내부 연산과 상태 업데이트 메커니즘은 다릅니다.

# 2. PyTorch를 이용해서 RNN 구현하기

## 2-1. nn.RNN()

In [ ]:
import torch
import torch.nn as nn

In [ ]:
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

In [ ]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 4, 5)
print('임의의 입력의 텐서 :')
print(inputs)

임의의 입력의 텐서 :
tensor([[[-8.9669e-20,  4.5653e-41, -5.5709e-40,  4.5653e-41, -8.9716e-20],
         [ 4.5653e-41, -1.0526e-38,  4.5653e-41, -8.9723e-20,  4.5653e-41],
         [-4.3782e+01,  4.5652e-41, -4.3782e+01,  4.5652e-41, -4.3731e+01],
         [ 4.5652e-41, -8.6399e-20,  4.5653e-41, -9.1319e-20,  4.5653e-41]]])


In [ ]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)

In [ ]:
outputs, hidden = cell(inputs)

In [ ]:
# 모든 time-step의 hidden_state
# 5개의 값이 4번 들어갔는데 출력은 8이 나옴 >> 5차원 > 8차원으로 변경됨됨 
print(outputs.shape)

torch.Size([1, 4, 8])


In [ ]:
# 첫번째 샘플의 모든 time-step의 hidden state
print(outputs)

tensor([[[-0.3999,  0.2703, -0.2019, -0.3700,  0.0727,  0.2130, -0.4390,
           0.0757],
         [-0.5890,  0.1464, -0.1237, -0.4859,  0.0540,  0.3119, -0.2035,
           0.1047],
         [ 1.0000, -0.9993,  1.0000, -0.3626, -1.0000,  0.7923,  1.0000,
           1.0000],
         [-0.2364,  0.8012, -0.0409,  0.2357,  0.4734,  0.2639, -0.8494,
           0.1282]]], grad_fn=<TransposeBackward1>)


In [ ]:
# 첫번째 샘플의 마지막 time-step의 hidden state
print(outputs[0][-1])

tensor([-0.2364,  0.8012, -0.0409,  0.2357,  0.4734,  0.2639, -0.8494,  0.1282],
       grad_fn=<SelectBackward0>)


In [ ]:
print(hidden.shape) # 최종 time-step의 hidden_state

torch.Size([1, 1, 8])


In [ ]:
# 마지막 시점의 hidden state
hidden[0]

tensor([[-0.2364,  0.8012, -0.0409,  0.2357,  0.4734,  0.2639, -0.8494,  0.1282]],
       grad_fn=<SelectBackward0>)

## 2-2. Deep RNN(Deep Recurrent Neural Network)

In [ ]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 4, 5)

In [ ]:
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
outputs, hidden = cell(inputs)

첫번째 리턴값의 크기는 층이 1개였던 RNN 셀 때와 달라지지 않았습니다. 여기서는 마지막 층의 모든 시점의 은닉 상태들입니다.



In [ ]:
# 첫번째 샘플의 모든 time-step의 hidden state
print(outputs.shape)

torch.Size([1, 4, 8])


두번째 리턴값의 크기는 층이 1개였던 RNN 셀 때와 달라졌는데, 여기서 크기는 (층의 개수, 배치 크기, 은닉 상태의 크기)에 해당됩니다.

In [ ]:
# (층의 개수, 배치 크기, 은닉 상태의 크기)
print(hidden.shape)

torch.Size([2, 1, 8])


## 2-3. Bidirectional Recurrent Neural Network

양방향 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 bidirectional에 값을 True로 전달하면 됩니다.

In [ ]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 4, 5)

In [ ]:
cell = nn.RNN(input_size = 5, hidden_size = 8,  batch_first=True, bidirectional = True)

In [ ]:
# outputs == 모든 시점의 은닉 상태
# hidden == 마지막 시점의 은닉 상태
outputs, hidden = cell(inputs)

첫번째 리턴값의 크기는 단뱡 RNN 셀 때보다 은닉 상태의 크기의 값이 두 배가 되었습니다. 여기서는 (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)의 크기를 가집니다. 이는 양방향의 은닉 상태 값들이 연결(concatenate)되었기 때문입니다.

In [ ]:
# (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)
print(outputs.shape)

torch.Size([1, 4, 16])


두번째 리턴값의 크기는 (2, 배치 크기, 은닉 상태의 크기)를 가집니다. 2가 의미하는 것은 하나는 순방향의 은닉 상태, 하나는 역방향의 은닉 상태이기 때문입니다.

In [ ]:
# (2, 배치 크기, 은닉 상태의 크기)
print(hidden.shape)

torch.Size([2, 1, 8])


In [ ]:
# 모든 시점의 concatenated 된 은닉 상태
print(outputs)

tensor([[[ 0.2306, -0.1377,  0.4939, -0.2032, -0.2539, -0.0260, -0.2801,
           0.0845,  0.0308,  0.0849,  0.0666, -0.1942,  0.1453, -0.0635,
           0.0184,  0.1507],
         [ 0.2505, -0.1488,  0.3406,  0.0613, -0.2234, -0.0775, -0.4335,
          -0.1326,  0.0290,  0.0717,  0.0457, -0.1997,  0.1334, -0.0724,
           0.0239,  0.1606],
         [ 0.2339, -0.1191,  0.3515, -0.0511, -0.1312,  0.0762, -0.4225,
          -0.0964,  0.0058,  0.0999,  0.0386, -0.2158,  0.0912, -0.0954,
           0.0494,  0.1192],
         [ 0.2322, -0.1485,  0.3492, -0.0027, -0.1432, -0.0102, -0.4366,
          -0.0907,  0.0578,  0.0553,  0.1336, -0.1830,  0.0929, -0.1182,
           0.0815,  0.0595]]], grad_fn=<TransposeBackward1>)


In [ ]:
# 순방향 기준 마지막 시점의 은닉 상태
print(hidden[0])

tensor([[ 0.2322, -0.1485,  0.3492, -0.0027, -0.1432, -0.0102, -0.4366, -0.0907]],
       grad_fn=<SelectBackward0>)


In [ ]:
# 역방향 기준 마지막 시점의 은닉 상태
print(hidden[1])

tensor([[ 0.0308,  0.0849,  0.0666, -0.1942,  0.1453, -0.0635,  0.0184,  0.1507]],
       grad_fn=<SelectBackward0>)
